# Creamos el dataset de respuestas entre usuarios

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
df = pd.read_pickle('../datasets/tweets_cleaned.pkl')

## Joineamos y mergeamos para conseguir un dataset que incluya, la pregunta, la respuesta y el partido politico de ambos

In [132]:
import pandas as pd
dfusers = pd.read_csv('../datasets/user_id_all.csv',names=["user_id","twitter_name","partido","tmp"],skiprows=1)
dfusers.drop(['tmp'],axis=1,inplace=True)

# Joineo informacion del que escribió el tweet
answers = df.merge(dfusers, how='left', left_on='user_id', right_on='user_id', suffixes=('','_tweeter'))

# Joineo quien fué el twitero original
answers = answers.merge(df.loc[df.is_reply==0,["conversation_id","user_id","full_text"]], how='left', left_on='conversation_id', right_on='conversation_id', suffixes=('','_head'))

# Joineo informacion del que escribió el tweet original
answers = answers.merge(dfusers, how='left', left_on='user_id_head', right_on='user_id', suffixes=('','_tweeter_head'))
answers.drop(['user_id_head'],axis=1,inplace=True)

#si desconozco quien fué le pongo anonimo
values = {'user_id_tweeter_head': 0, 'partido_tweeter_head': 'desconocido', 'twitter_name_tweeter_head': 'anonimo'}
answers.fillna(value=values,inplace=True)
answers["user_id_tweeter_head"]=answers["user_id_tweeter_head"].astype(np.int64)

# Agrego columna para indicar si es la misma persona o partido
answers["same_user_id"] = answers["user_id"]==answers["user_id_tweeter_head"]
answers["same_partido"] = answers["partido"]==answers["partido_tweeter_head"]

# Borro todos los tweets de personas no identificadas
answers.drop(answers.loc[answers.partido.isna()].index,axis=0,inplace=True)

## Revisamos la cantidad de datos y algunos ejemplos

In [139]:
print("Tweets totales de personas con partido: *{}*".format(len(answers)))
print("*{}* son tweets Originales y *{}* son respuestas ".format(len(answers.loc[(answers.is_reply==0)]),len(answers.loc[(answers.is_reply==1)])))
print("De las respuestas, solo *{}* son entre partidos distintos ".format(len(answers.loc[(answers.is_reply==1) & ~(answers.partido==answers.partido_tweeter_head)])))

print("Respuestas entre partidos:")

display(answers.loc[(answers.is_reply==1) & ~(answers.partido==answers.partido_tweeter_head)].pivot_table(index="partido",columns="partido_tweeter_head",values="full_text",aggfunc="count"))
    
print("Se puede apreciar que la interaccion entre partidos es muy baja. ")

print("")
print("")
print("")

print("Ejemplos de tweets del FRENTE de TODOS hacia JUNTOS POR EL CAMBIO:")
display(answers.loc[(answers.is_reply==1) & (answers.partido=="FRENTE DE TODOS") & (answers.partido_tweeter_head=="JUNTOS POR EL CAMBIO"),
                    ["twitter_name","full_text","twitter_name_tweeter_head","full_text_head"]].sample(5))


print("Ejemplos de tweets del JUNTOS POR EL CAMBIO hacia FRENTE de TODOS:")
display(answers.loc[(answers.is_reply==1) & (answers.partido=="JUNTOS POR EL CAMBIO") & (answers.partido_tweeter_head=="FRENTE DE TODOS"),
                    ["twitter_name","full_text","twitter_name_tweeter_head","full_text_head"]].sample(5))


Tweets totales de personas con partido: *597864*
*507210* son tweets Originales y *90654* son respuestas 
De las respuestas, solo *28077* son entre partidos distintos 
Respuestas entre partidos:


partido_tweeter_head,FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES,FRENTE DE TODOS,JUNTOS POR EL CAMBIO,PTS - FRENTE DE IZQUIERDA,desconocido
partido,,,,,
CONSENSO FEDERAL,NaN,NaN,NaN,NaN,14.0
FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES,NaN,2.0,NaN,NaN,4.0
FRENTE DE TODOS,1.0,NaN,684.0,55.0,18821.0
JUNTOS POR EL CAMBIO,NaN,1162.0,NaN,12.0,6508.0
PTS - FRENTE DE IZQUIERDA,2.0,148.0,8.0,NaN,649.0
UNIDAD FEDERAL PARA EL DESARROLLO,NaN,NaN,NaN,NaN,7.0


Se puede apreciar que la interaccion entre partidos es muy baja. 



Ejemplos de tweets del FRENTE de TODOS hacia JUNTOS POR EL CAMBIO:


,twitter_name,full_text,twitter_name_tweeter_head,full_text_head
149267,SUSANAOTTA,@Jofa2010 Votan a Consciencia!!!!\nVotan por un PAÍS!!!\nVotan por una REPÚBLICA!!!!\n Tienen memoria!!!!,JOFA2010,"En Córdoba también hay crisis, hay desempleo , la guita que no alcanza para llegar a fin de mes etc. Pero en Córdoba no votan a ladrones y mentirosos. Muchas Gracias por dar el ejemplo otra vez! Felicitaciones Cordobeses!\nMuchas gracias nos vemos el 27/10 🥂."
11778,PATRICIODIPALMA,"@Juancmarino Felicitaciones y un gran logro para q este país empiece un camino de cambio y unidad entre todos,fuerza y a seguir haciendo cosas tan importantes.\nMis felicitaciones a vos y los papás de Justina. Fuerte ABRAZO.",JUANCMARINO,"SE PROMULGÓ LA LEY JUSTINA\nDespués de muchos meses de trabajo estoy feliz de que esta Ley ya sea una realidad. Hoy es un día histórico, el comienzo de un nuevo capítulo en la donación de órganos de nuestro país en donde se salvarán cada día más vidas.\nhttps://t.co/2VpdVcZ7WC"
608631,ANDRESMALAMUD,"@JosefaMendo @diputadosucr #SeráLey, querida Jose: para nosotros, para nuestra posteridad y para todas las mujeres del mundo que quieran habitar en el suelo argentino 💚",JOSEFAMENDO,"Ese día pude citar a mi amigo @andresmalamud, citar el manifiesto liminar y votar por algo que hace años milito.\nEse día dije que si no salía, nadie se rendía, íbamos a seguir haciendo ruido porque la historia nunca se escribió en silencio. #SeraLey 💚\nhttps://t.co/ugR6uKNrs3 https://t.co/yzjWihwhfQ"
559708,OSVALDOADRIANV3,"@PatoBullrich Pato, lo único que perseguiste vos durante tu gestión, fue a un pibe que estaba cortando una ruta. Y por arte de magia apareció ahogado en un arroyo de 35cm.",PATOBULLRICH,"El gobierno derogó un protocolo que instruía a las FF a perseguir narcos y trata por un ciberpatrullaje ideológico, que en vez de perseguir a delincuentes irá tras el “humor social” de la ciudadanía. Dijeron que no lo iban a hacer y lo están haciendo. https://t.co/f134dyV1W5"
370724,JUMASERRANO,"@TadeoGZ En serio Tadeo, no hay ninguna necesidad de agitar un conflicto donde hay una explicación razonable desde lo sanitario y desde la optimización de los escasos recursos disponibles. Responsabilidad por favor.",TADEOGZ,El ministro @ginesggarcia dice que la distribución de los recursos se hace con Equidad. A Mendoza le bloquearon respiradores y reactivos para hacer el test #Covid_19 y seguimos esperando la Tarjeta Alimentaria🤔😷\n#DiosAtiendeenBuenosAires https://t.co/Te1lyli9xe vía @elsolonline


Ejemplos de tweets del JUNTOS POR EL CAMBIO hacia FRENTE de TODOS:


,twitter_name,full_text,twitter_name_tweeter_head,full_text_head
968485,MARFABSAI,@pviloutaoficial @_patitom Ese es el gobierno que uds los periodistas bancan!!,PVILOUTAOFICIAL,Mario Ishii: “Yo los tengo que cubrir cuando están vendiendo falopa con las ambulancias” - Infobae https://t.co/MhHgFiAe4G
996777,TOMAASSANTOS,"@alferdez @MovimientoEvita Que esencial, che",ALFERDEZ,"Hoy participé del homenaje a Eva Perón junto a compañeros y compañeras del @MovimientoEvita, que realizaron 10.000 ollas populares. Les agradezco el trabajo silencioso que hacen en cada rincón del país.\n\nComo peronistas, nuestro compromiso es con los que menos tienen. https://t.co/2ojyS7j7zT"
835590,DEFIENDOVALORES,@Luis_Delia Q alegria! Ahora vas a poder decir q te curaste en manos de médicos oligarcas en barrio oligarca en sanatorio ultra oligarca!\nTe salva el capitalismo Luis! Acordate de eso siempre!,LUIS_DELIA,Tantas veces me mataron...\nTantas veces me morí...\nSin embargo estoy aquí\nResucitando..✌🏽✌🏽✌🏽✌🏽✌🏽
919330,JORGENORBERTO15,"@LaBelgrana @laventanadjulia Y de un número indeterminado de nuevos empleados estatales, algunos juicios en Ciadi, y 450mil nuevos juicios de jubilados",LABELGRANA,"El peronismo no pagará su deuda y pateará vencimientos x el próximo gobierno. El próximo gno. deberá hacerse cargo de la deuda peronista, el atraso cambiario peronista, la emisión peronista, el déficit peronista y las tarifas atrasadas peronistas. Loop infinito de peronismo."
902851,GUSTAVOSRUBIO,@SergioBerniArg Consecuencias del desarme civil https://t.co/kWOm3fN9aC,SERGIOBERNIARG,El uso de la legítima defensa tiene ciertos parámetros que se tendrían que contemplar en el caso del señor Ríos de Quilmes: pusieron en riesgo su vida y actuó en una situación extrema.


# Creamos los archivos para trabajar con el GPT2

In [175]:
def clean(text):
    return text

def saveToTrainGPT2(writePath,df):
    dataset=''
    df["full_text_head"] = df["full_text_head"].replace("  ","").replace(r'http\S+|t\.co\S+', 'URL',regex=True).replace('\\n', ' ').replace('\n', ' ')
    df["full_text"] = df["full_text"].replace("  ","").replace(r'http\S+|t\.co\S+', 'URL',regex=True).replace('\\n', ' ').replace('\n', ' ')
    for i,row in df.iterrows():
        dataset += "<|endoftext|>[POST] " + clean(row.full_text_head) + "\n[RESPONSE] " + clean(row.full_text) + '\n'
    f = open(writePath,'w')
    f.write(dataset)
    print("Archivo " + writePath + " Creado")
    

fdt_to_jpc=answers.loc[(answers.is_reply==1) & (answers.partido=="FRENTE DE TODOS") & (answers.partido_tweeter_head=="JUNTOS POR EL CAMBIO"),["full_text_head","full_text"]]
saveToTrainGPT2('../datasets/FDT_to_JPC.txt',fdt_to_jpc)
    
    
jpc_to_fdt=answers.loc[(answers.is_reply==1) & (answers.partido=="JUNTOS POR EL CAMBIO") & (answers.partido_tweeter_head=="FRENTE DE TODOS"),["full_text_head","full_text"]]
saveToTrainGPT2('../datasets/JPC_to_FDT.txt',jpc_to_fdt)